### Authenticate GEE and start Geemap

In [7]:
#!pip install geemap
import ee
ee.Initialize()
#authenticate ee
import geemap

### outline plot

In [8]:
# Make an area of interest geometry centered on San Francisco.
gulf_coast = ee.Geometry.BBox(-98, 26,-82,31)

# Import a Landsat 8 image, subset the thermal band, and clip to the

# Define a map centered on Gulf Coast
Map = geemap.Map()

Map.setCenter(-90,28,7)
Map.addLayer(gulf_coast, {'color': 'red'}, 'bounding box')
Map.addLayerControl() 
Map

Map(center=[28, -90], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

### load delta binary map

In [10]:
merit = ee.Image("MERIT/DEM/v1_0_3").clip(gulf_coast);
merit = merit.reproject(crs=merit.projection(), scale=100)
dmask = merit.lt(5).selfMask()


WaterMask = ee.ImageCollection('MODIS/006/MOD44W').filter(ee.Filter.date('2015-01-01', '2015-05-01'))
WaterMask = ee.Image(WaterMask.first()).select('water_mask').eq(0).convolve(ee.Kernel.square(1)).gt(0)
dmask = dmask.updateMask(WaterMask)

Map = geemap.Map()
Map.setCenter(-90,28,7)
Map.addLayer(dmask)
Map

Map(center=[28, -90], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

## label deltas

Labeling objects is often the first step in object analysis. Here, the `connectedComponents()` function is used to identify image objects and assign a unique ID to each; all pixels belonging to an object are assigned the same integer ID value. The result is a copy of the input image with an additional "labels" band associating pixels with an object ID value based on connectivity of pixels in the first band of the image.

In [11]:
# Uniquely label the hotspot image objects.
object_id = dmask.connectedComponents(connectedness=ee.Kernel.plus(1), maxSize=1024)

Map = geemap.Map()
Map.setCenter(-90,28,7)

# Add the uniquely ID'ed objects to the map.
# Map.addLayer(object_id.randomVisualizer(), {}, 'Objects')

# Compute the number of pixels in each object defined by the "labels" band.
object_size = object_id.select('labels').connectedPixelCount(maxSize=1024, eightConnected=False)
#Map.addLayer(object_size, {}, 'Object n pixels')

# Get a pixel area image.
pixel_area = ee.Image.pixelArea()

# Multiply pixel area by the number of pixels in an object to calculate
# the object area. The result is an image where each pixel
# of an object relates the area of the object in m^2.
object_area = object_size.multiply(pixel_area)

# Add the object area to the map.
#Map.addLayer(object_area,{'min': 0, 'max': 1e8, 'palette': ['red', 'green']},'Object area m^2')

# Update the mask of the `object_id` layer defined previously using the
# minimum area mask just defined.#only add deltas larger than 10km2
object_id = object_id.updateMask(object_area.gte(1e7))
Map.addLayer(object_id, {}, 'Large deltas')

# Calculate the mean temperature per object defined by the previously added
# "labels" band.
# delta = ee.Image(0).addBands(object_id.select('labels'))
#delta_comp = delta.reduceConnectedComponents(
#    reducer=ee.Reducer.mean(), labelBand='labels')

# Add object mean temperature to the map and display it.
Map


Map(center=[28, -90], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

## reduce to vectors
reduce to vectors

In [13]:
delta_outlines = object_id.select('labels').reduceToVectors(**{
  'geometry': gulf_coast,
  'crs': object_id.projection(),
  'scale': 1000,
  'geometryType': 'polygon',
  'eightConnected': False,
})
Map = geemap.Map()
Map.addLayer(delta_outlines)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Join with delta database

Join earlier dataset with delta dataset, trying a spatial join


In [6]:
Map = geemap.Map()
Map.setCenter(-90,28,7)

GlobalDeltaData = ee.FeatureCollection('users/jhnienhuis/GlobalDeltaBasins_app_shp');
DeltaMouth = ee.Geometry.MultiPoint(GlobalDeltaData.aggregate_array("MouthLon").zip(GlobalDeltaData.aggregate_array("MouthLat")))

DeltaMouthBuffer = ee.FeatureCollection(DeltaMouth).map(lambda f: f.buffer(2000))

join_filter = ee.Filter.withinDistance(2000, '.geo', None, '.geo')
DeltaMatch = ee.Join.simple().apply(delta_outlines, DeltaMouthBuffer, join_filter)

#Map.addLayer(DeltaMouthBuffer, {'color': 'b0b0b0'}, "Mouths")
Map.addLayer(DeltaMatch, {}, "Outlines")



Map.addLayer(DeltaMouth)
Map

Map(center=[28, -90], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…